In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from datetime import date, timedelta

In [2]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import mlflow
import dagshub

print("All imports successful!")

All imports successful!


In [6]:
file_path = '../data/kings.xlsx'
df = pd.read_excel(file_path)

In [4]:
# Convertir variables categóricas en numéricas usando LabelEncoder
le_team = LabelEncoder()

In [7]:
df['team1'] = le_team.fit_transform(df['team1'])
df['team2'] = le_team.fit_transform(df['team2'])

In [8]:
df

,match_id,date,team1,team2,goals_team1,goals_team2,dado_number,minutes_after_dice_roll,goals_team1_after_dice,goals_team2_after_dice,Total_goals_in_dice
0,S1J3P1,2023-01-15,3,0,0,2,4,3,0,2,2
1,S1J3P2,2023-01-15,7,11,0,1,4,2,0,0,0
2,S1J3P3,2023-01-15,8,10,1,0,4,2,0,0,0
3,S1J3P4,2023-01-15,4,6,0,2,5,2,0,0,0
4,S1J3P5,2023-01-15,9,5,0,2,4,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
303,S4J11P5,2024-12-08,1,4,4,1,3,4,1,1,2
304,S4J11P6,2024-12-08,8,3,4,2,2,4,2,2,4
305,S4Q4P1,2024-12-13,4,6,2,1,3,3,0,1,1
306,S4Q4P2,2024-12-13,11,8,1,0,3,3,1,1,2


In [9]:
# Definir las variables predictoras (features) y las variables objetivo (targets)
X = df[['team1', 'team2', 'goals_team1', 'goals_team2', 'dado_number', 'minutes_after_dice_roll']]
y_team1 = df['goals_team1_after_dice']
y_team2 = df['goals_team2_after_dice']

In [10]:
X_train, X_test, y_team1_train, y_team1_test, y_team2_train, y_team2_test = train_test_split(X, y_team1, y_team2, test_size=0.2, random_state=42)

# Usar Random Forest para entrenar ambos modelos de goles
model_team1 = RandomForestRegressor()
model_team2 = RandomForestRegressor()

In [11]:
model_team1.fit(X_train, y_team1_train)
model_team2.fit(X_train, y_team2_train)

RandomForestRegressor()

In [12]:
y_team1_pred = model_team1.predict(X_test)
y_team2_pred = model_team2.predict(X_test)

In [13]:
# Calcular el error medio cuadrado para evaluar el rendimiento
mse_team1 = mean_squared_error(y_team1_test, y_team1_pred)
mse_team2 = mean_squared_error(y_team2_test, y_team2_pred)

print(f'Error medio cuadrado (MSE) para goles del equipo 1: {mse_team1}')
print(f'Error medio cuadrado (MSE) para goles del equipo 2: {mse_team2}')

Error medio cuadrado (MSE) para goles del equipo 1: 0.7661203405017922
Error medio cuadrado (MSE) para goles del equipo 2: 0.656326097670251


In [14]:
dagshub.init(url="https://dagshub.com/PacoTinoco/KingsLeagueML", mlflow=True)

MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

print(MLFLOW_TRACKING_URI)

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="randomforestKL")

Accessing as PacoTinoco

Initialized MLflow to track repo "PacoTinoco/KingsLeagueML"

Repository PacoTinoco/KingsLeagueML initialized!

https://dagshub.com/PacoTinoco/KingsLeagueML.mlflow


2024/12/19 15:43:50 INFO mlflow.tracking.fluent: Experiment with name 'randomforestKL' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/2033e239dca8402ea4a54a141a729a8f', creation_time=1734644630884, experiment_id='0', last_update_time=1734644630884, lifecycle_stage='active', name='randomforestKL', tags={}>

In [15]:
# Definir X e y
X = df[['team1', 'team2', 'goals_team1', 'goals_team2', 'dado_number', 'minutes_after_dice_roll']]
y_team1 = df['goals_team1_after_dice']
y_team2 = df['goals_team2_after_dice']
from sklearn.model_selection import train_test_split

# Dividir en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_team1_train, y_team1_test, y_team2_train, y_team2_test = train_test_split(X, y_team1, y_team2, test_size=0.2, random_state=42)


In [18]:
mlflow.sklearn.autolog()

def objective_rf(params):
    with mlflow.start_run(nested=True):
        # Set model tag
        mlflow.set_tag("model_family", "random_forest")
        
        # Log parameters
        mlflow.log_params(params)
        
        # Train RandomForest model
        rf_model = RandomForestRegressor(
            n_estimators=int(params['n_estimators']),
            max_depth=int(params['max_depth']),
            min_samples_split=int(params['min_samples_split']),
            min_samples_leaf=int(params['min_samples_leaf']),
            random_state=42
        )
        rf_model.fit(X_train, y_team1_train)
        
        # Predict on validation dataset
        y_pred = rf_model.predict(X_test)
        
        # Calculate RMSE
        rmse = np.sqrt(mean_squared_error(y_team1_test, y_pred))
        
        # Log RMSE metric
        mlflow.log_metric("rmse", rmse)
        
        return {'loss': rmse, 'status': STATUS_OK}

# Define search space for RandomForest
search_space_rf = {
    'n_estimators': hp.quniform('n_estimators', 50, 100, 1),
    'max_depth': hp.quniform('max_depth', 5, 15, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 5, 1),
    'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 2, 1),
}


# Run hyperparameter optimization
with mlflow.start_run(run_name="Parent Random Forest", nested=True):
    best_params_rf = fmin(
        fn=objective_rf,
        space=search_space_rf,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    
    # Log best parameters
    mlflow.log_params(best_params_rf)

2024/12/19 15:49:53 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 0.24.1 <= scikit-learn <= 1.5.1, but the installed version is 1.5.2. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.


  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

2024/12/19 15:49:53 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\PycharmProjects\Proyecto_Final_CDD\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."

2024/12/19 15:49:54 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\

 10%|█         | 1/10 [00:07<01:04,  7.13s/trial, best loss: 0.8569376862394789]

2024/12/19 15:50:03 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\PycharmProjects\Proyecto_Final_CDD\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."

2024/12/19 15:50:07 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\

...

2024/12/19 15:52:02 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\PycharmProjects\Proyecto_Final_CDD\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."

2024/12/19 15:52:06 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\

 90%|█████████ | 9/10 [02:22<00:16, 16.80s/trial, best loss: 0.8069182027619816]

2024/12/19 15:52:19 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\PycharmProjects\Proyecto_Final_CDD\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."

2024/12/19 15:52:23 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\

100%|██████████| 10/10 [02:40<00:00, 16.00s/trial, best loss: 0.8069182027619816]


2024/12/19 15:52:35 INFO mlflow.tracking._tracking_service.client: 🏃 View run Parent Random Forest at: https://dagshub.com/PacoTinoco/KingsLeagueML.mlflow/#/experiments/0/runs/6034c65dbab041dab7e8dcda219e0850.
2024/12/19 15:52:35 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/PacoTinoco/KingsLeagueML.mlflow/#/experiments/0.


In [19]:
####

# Autolog de MLflow
mlflow.sklearn.autolog()

# Función objetivo
def objective_rf(params, X_train, y_train, X_test, y_test, target_name):
    with mlflow.start_run(nested=True):
        # Etiqueta de modelo
        mlflow.set_tag("model_family", "random_forest")
        mlflow.set_tag("target", target_name)
        
        # Registrar hiperparámetros
        mlflow.log_params(params)
        
        # Entrenar el modelo
        rf_model = RandomForestRegressor(
            n_estimators=int(params['n_estimators']),
            max_depth=int(params['max_depth']),
            min_samples_split=int(params['min_samples_split']),
            min_samples_leaf=int(params['min_samples_leaf']),
            random_state=42
        )
        rf_model.fit(X_train, y_train)
        
        # Predecir y calcular RMSE
        y_pred = rf_model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        
        # Registrar métrica
        mlflow.log_metric("rmse", rmse)
        
        return {'loss': rmse, 'status': STATUS_OK}

# Espacio de búsqueda
search_space_rf = {
    'n_estimators': hp.quniform('n_estimators', 50, 100, 1),
    'max_depth': hp.quniform('max_depth', 5, 15, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 5, 1),
    'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 2, 1),
}

# Optimización para team1 y team2
results = {}
for target_name, y_train, y_test in [('team1', y_team1_train, y_team1_test), ('team2', y_team2_train, y_team2_test)]:
    with mlflow.start_run(run_name=f"Parent Random Forest {target_name}"):
        best_params_rf = fmin(
            fn=lambda params: objective_rf(params, X_train, y_train, X_test, y_test, target_name),
            space=search_space_rf,
            algo=tpe.suggest,
            max_evals=10,
            trials=Trials()
        )
        
        # Log best parameters
        mlflow.log_params(best_params_rf)
        
        # Guardar resultados
        results[target_name] = best_params_rf

# Mostrar resultados
print("Best parameters for team1:", results['team1'])
print("Best parameters for team2:", results['team2'])

2024/12/19 16:03:11 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 0.24.1 <= scikit-learn <= 1.5.1, but the installed version is 1.5.2. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.


  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

2024/12/19 16:03:12 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\PycharmProjects\Proyecto_Final_CDD\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."

2024/12/19 16:03:13 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\

 10%|█         | 1/10 [00:07<01:11,  7.96s/trial, best loss: 0.8177729973085064]

2024/12/19 16:03:23 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\PycharmProjects\Proyecto_Final_CDD\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."

2024/12/19 16:03:28 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\

...

 90%|█████████ | 9/10 [02:41<00:17, 17.87s/trial, best loss: 0.7444837095796192]

2024/12/19 16:08:52 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\PycharmProjects\Proyecto_Final_CDD\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."

2024/12/19 16:08:56 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\

100%|██████████| 10/10 [03:00<00:00, 18.00s/trial, best loss: 0.7444837095796192]


2024/12/19 16:09:07 INFO mlflow.tracking._tracking_service.client: 🏃 View run Parent Random Forest team2 at: https://dagshub.com/PacoTinoco/KingsLeagueML.mlflow/#/experiments/0/runs/9a00c06243d944d5acaec5901cc905c5.
2024/12/19 16:09:07 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/PacoTinoco/KingsLeagueML.mlflow/#/experiments/0.


Best parameters for team1: {'max_depth': 9.0, 'min_samples_leaf': 2.0, 'min_samples_split': 4.0, 'n_estimators': 58.0}
Best parameters for team2: {'max_depth': 6.0, 'min_samples_leaf': 2.0, 'min_samples_split': 3.0, 'n_estimators': 78.0}


In [20]:
best_params_rf

{'max_depth': 6.0,
 'min_samples_leaf': 2.0,
 'min_samples_split': 3.0,
 'n_estimators': 78.0}

### Registrar el champion

In [21]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="RF-model"
)

Successfully registered model 'RF-model'.
2024/12/19 16:42:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RF-model, version 1
Created version '1' of model 'RF-model'.


In [22]:
from datetime import datetime
from mlflow import MlflowClient

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.update_registered_model(
    name="RF-model",
    description="Model registry for the Kings League prediction",
)

new_alias = "champion"
date = datetime.today()
model_version = "1"

# create "champion" alias for version 1 of model "nyc-taxi-model"
client.set_registered_model_alias(
    name="RF-model",
    alias=new_alias,
    version=model_version
)

client.update_model_version(
    name="RF-model",
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

<ModelVersion: aliases=['champion'], creation_timestamp=1734648154200, current_stage='None', description='The model version 1 was transitioned to champion on 2024-12-19 16:43:14.900652', last_updated_timestamp=1734648195002, name='RF-model', run_id='beeefb83319445aea9f40cb4da64dafd', run_link='', source='mlflow-artifacts:/2033e239dca8402ea4a54a141a729a8f/beeefb83319445aea9f40cb4da64dafd/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [23]:
import mlflow.pyfunc

model_name = "RF-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_version = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

champion_version.predict(X_test)

array([0.72359849, 0.77369538, 0.46915542, 1.0687776 , 0.77703764,
       1.62081394, 0.51409534, 2.63626682, 1.07523092, 0.31952229,
       0.56079907, 1.82600902, 0.76573395, 1.00524547, 0.49587599,
       0.3889113 , 0.97670931, 0.81282085, 0.83144481, 0.67829423,
       0.54007902, 0.93225235, 0.96581681, 1.41325927, 1.67875773,
       0.1991636 , 1.11237682, 0.82488944, 0.1378478 , 1.43211685,
       0.27080466, 0.03580831, 1.10548794, 0.64466188, 0.7836858 ,
       0.79049706, 0.60827566, 0.3869063 , 1.01778844, 0.68489921,
       0.26879569, 1.57778257, 0.06977524, 0.22134721, 0.26604418,
       0.09329361, 0.91184398, 0.88094882, 0.5352479 , 0.44549469,
       0.26966155, 0.0804731 , 0.69659348, 0.41330609, 0.00950812,
       0.73231262, 1.05396798, 0.12677739, 0.91252747, 0.75226271,
       1.22467141, 2.2933396 ])

In [30]:
import pathlib
# Crear un directorio para guardar el modelo, si no existe
pathlib.Path("models").mkdir(exist_ok=True)

In [32]:
import pickle

# Guarda el modelo en un archivo .pkl
with open("pythonProject22/KingsLeagueML/training_pipeline/models/kings_league_rf_model.pkl", "wb") as f_model:
    pickle.dump(champion_version, f_model)

FileNotFoundError: [Errno 2] No such file or directory: 'pythonProject22/KingsLeagueML/training_pipeline/models/kings_league_rf_model.pkl'